In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
from pydicom import dcmread

from skimage.transform import resize
import os
import shutil

def read_meta_data(filename):
    dataset = pd.read_csv(filename)
    return dataset

In [2]:
##Read CBIS_DDSM metadata from your local folder.

cbisddsm = read_meta_data("/home/careinfolab/StanfordMammogram/metadata.csv")

In [3]:
## Read other CBIS-DDSM metadata files that contains benign and malignant labels. 
def MalignOrBenign(DistinctName):
    cal_case_train_set = read_meta_data("/home/careinfolab/StanfordMammogram/calc_case_description_train_set.csv")
    cal_case_test_set = read_meta_data("/home/careinfolab/StanfordMammogram/calc_case_description_test_set.csv")
    mass_case_train_set = read_meta_data("/home/careinfolab/StanfordMammogram/mass_case_description_test_set.csv")
    mass_case_test_set = read_meta_data("/home/careinfolab/StanfordMammogram/mass_case_description_train_set.csv")

    cal_case_train = cal_case_train_set[cal_case_train_set["image file path"].str.contains(DistinctName)]
    cal_case_test = cal_case_test_set[cal_case_test_set["image file path"].str.contains(DistinctName)]
    mass_case_train = mass_case_train_set[mass_case_train_set["image file path"].str.contains(DistinctName)]
    mass_case_test = mass_case_test_set[mass_case_test_set["image file path"].str.contains(DistinctName)]

    if cal_case_train.shape[0] > 0:
        label = cal_case_train['pathology'].values.astype('str')[0]
        return label

    if cal_case_test.shape[0] > 0:
        label = cal_case_test['pathology'].values.astype('str')[0]
        return label

    if mass_case_train.shape[0] > 0:
        label = mass_case_train['pathology'].values.astype('str')[0]
        return label

    if mass_case_test.shape[0] > 0:
        label = mass_case_test['pathology'].values.astype('str')[0]
        return label

    return "No_Label"

In [4]:
## Method to convert from DICOM to PNGs
def ConvertFromDCMtoPNG(srcPath, dstFolderPath):
    im_dim_x = 800
    im_dim_y = 600

    im = pydicom.dcmread(srcPath)
    print(srcPath + "'s resolution is " + str(im.pixel_array.shape))
    im = im.pixel_array.astype(float)
## anti aliasing is true. It is resized into 800 * 600. 
    im = resize(im, (im_dim_x, im_dim_y), anti_aliasing=True)

## Rescaled into gray scale image. 
    rescaled_image = (np.maximum(im, 0)/im.max())*65536

    final_image= np.uint16(rescaled_image)

    print(srcPath + "'s resolution rescaled to " + str(final_image.shape))


    final_image = Image.fromarray(final_image)
    final_image.save(dstFolderPath)

In [6]:
PositiveCBISDF = []
NegativeCBISDF = []
### Get labels for each mammogram and copy them into a separate local folder and create their metadata files. 
def RegenerateUIDandPath(dataset):
    for idx in dataset.index:
        lastChar = dataset['Subject ID'][idx][-1]
        if lastChar.isnumeric():
            print("the last character is numeric. ANd is going to be continued!")
            continue
        
        distinctiveName = dataset['Subject ID'][idx]
        distinctiveUID = dataset['Series UID'][idx]
        distinctiveRow = dataset.iloc[idx]
        ## 1-1 is the breast image
        RawPath = dataset['File Location'][idx] + "//1-1.dcm"
        ## File path is changed to the local file path
        NewPathSrc = "/home/careinfolab/StanfordMammogram" + RawPath[1:len(RawPath)]
        
        NewPositivePath = "/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/pos/"
        NewNegativePath = "/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/neg/"
        if not os.path.exists(NewPositivePath):
        # if the demo_folder directory is not present
        # then create it.
            os.makedirs(NewPositivePath)
            print("Positive path is created!")
            
        if not os.path.exists(NewNegativePath):
        # if the demo_folder directory is not present
        # then create it.
            os.makedirs(NewNegativePath)
            print("Negative path is created!")
        
        label = MalignOrBenign(distinctiveName)
        print(NewPathSrc + " is : " + label)
        if 'benign' in label.lower():
            destFolderPath =  NewNegativePath + str(idx) + ".png"
            NegativeCBISDF.append([distinctiveUID, destFolderPath])
            print("Benign :" + str(destFolderPath))
            print('label :' + label)
            
        else:
            destFolderPath =  NewPositivePath + str(idx) + ".png"
            PositiveCBISDF.append([distinctiveUID, destFolderPath])
            print("Malignant : " + str(destFolderPath))
            print('label :' + label)
        ConvertFromDCMtoPNG(NewPathSrc, destFolderPath)

In [7]:
RegenerateUIDandPath(cbisddsm)

the last character is numeric. ANd is going to be continued!
Positive path is created!
Negative path is created!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00038_LEFT_CC/08-29-2017-DDSM-NA-96009/1.000000-full mammogram images-63992//1-1.dcm is : BENIGN
Benign :/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/neg/1.png
label :BENIGN
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00038_LEFT_CC/08-29-2017-DDSM-NA-96009/1.000000-full mammogram images-63992//1-1.dcm's resolution is (4616, 3016)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00038_LEFT_CC/08-29-2017-DDSM-NA-96009/1.000000-full mammogram images-63992//1-1.dcm's resolution rescaled to (800, 600)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO/08-29-2017-DDSM-NA-17613/1.000000-full mammogram images-97934//1-1.dcm is : BENIGN
Benign :/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/neg/2.png
label :BENIGN
/home/car

/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00100_RIGHT_MLO/08-29-2017-DDSM-NA-80242/1.000000-full mammogram images-61099//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00127_RIGHT_CC/08-29-2017-DDSM-NA-48220/1.000000-full mammogram images-81950//1-1.dcm is : MALIGNANT
Malignant : /home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/pos/29.png
label :MALIGNANT
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00127_RIGHT_CC/08-29-2017-DDSM-NA-48220/1.000000-full mammogram images-81950//1-1.dcm's resolution is (6706, 3796)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00127_RIGHT_CC/08-29-2017-DDSM-NA-48220/1.000000-full mammogram images-81950//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00127_RIGHT

/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00163_LEFT_MLO/08-29-2017-DDSM-NA-60281/1.000000-full mammogram images-62045//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00164_RIGHT_CC/08-29-2017-DDSM-NA-77973/1.000000-full mammogram images-58442//1-1.dcm is : MALIGNANT
Malignant : /home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/pos/56.png
label :MALIGNANT
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00164_RIGHT_CC/08-29-2017-DDSM-NA-77973/1.000000-full mammogram images-58442//1-1.dcm's resolution is (4741, 2506)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00164_RIGHT_CC/08-29-2017-DDSM-NA-77973/1.000000-full mammogram images-58442//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00180_LEFT_C

/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00223_LEFT_MLO/08-29-2017-DDSM-NA-14373/1.000000-full mammogram images-41038//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00257_RIGHT_CC/08-29-2017-DDSM-NA-82396/1.000000-full mammogram images-47230//1-1.dcm is : BENIGN
Benign :/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/neg/83.png
label :BENIGN
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00257_RIGHT_CC/08-29-2017-DDSM-NA-82396/1.000000-full mammogram images-47230//1-1.dcm's resolution is (4704, 2720)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00257_RIGHT_CC/08-29-2017-DDSM-NA-82396/1.000000-full mammogram images-47230//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00246_RIGHT_MLO/08-29-

/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00344_LEFT_MLO/08-29-2017-DDSM-NA-86499/1.000000-full mammogram images-42263//1-1.dcm's resolution rescaled to (800, 600)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00352_LEFT_CC/08-29-2017-DDSM-NA-56354/1.000000-full mammogram images-56814//1-1.dcm is : BENIGN
Benign :/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/neg/107.png
label :BENIGN
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00352_LEFT_CC/08-29-2017-DDSM-NA-56354/1.000000-full mammogram images-56814//1-1.dcm's resolution is (4496, 2880)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00352_LEFT_CC/08-29-2017-DDSM-NA-56354/1.000000-full mammogram images-56814//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Calc-Test_P_00352_LEFT_MLO/08-29-2017-DDSM-NA-94204/1.000000-full mammogram images-22939//1-1.dcm

...

/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01817_RIGHT_MLO/07-20-2016-DDSM-NA-53241/1.000000-full mammogram images-01358//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01819_RIGHT_CC/07-20-2016-DDSM-NA-37375/1.000000-full mammogram images-55782//1-1.dcm is : BENIGN_WITHOUT_CALLBACK
Benign :/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/neg/6653.png
label :BENIGN_WITHOUT_CALLBACK
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01819_RIGHT_CC/07-20-2016-DDSM-NA-37375/1.000000-full mammogram images-55782//1-1.dcm's resolution is (5648, 4040)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01819_RIGHT_CC/07-20-2016-DDSM-NA-37375/1.000000-full mammogram images-55782//1-1.dcm's resolution rescaled to (800, 600)
the last character is numer

/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01831_RIGHT_MLO/07-20-2016-DDSM-NA-74905/1.000000-full mammogram images-85467//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01840_RIGHT_MLO/07-20-2016-DDSM-NA-89949/1.000000-full mammogram images-98857//1-1.dcm is : BENIGN
Benign :/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/neg/6679.png
label :BENIGN
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01840_RIGHT_MLO/07-20-2016-DDSM-NA-89949/1.000000-full mammogram images-98857//1-1.dcm's resolution is (5491, 2296)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01840_RIGHT_MLO/07-20-2016-DDSM-NA-89949/1.000000-full mammogram images-98857//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continue

/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01850_RIGHT_MLO/07-20-2016-DDSM-NA-89673/1.000000-full mammogram images-18759//1-1.dcm's resolution rescaled to (800, 600)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01853_LEFT_MLO/07-20-2016-DDSM-NA-06263/1.000000-full mammogram images-30361//1-1.dcm is : MALIGNANT
Malignant : /home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/pos/6702.png
label :MALIGNANT
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01853_LEFT_MLO/07-20-2016-DDSM-NA-06263/1.000000-full mammogram images-30361//1-1.dcm's resolution is (5476, 3451)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01853_LEFT_MLO/07-20-2016-DDSM-NA-06263/1.000000-full mammogram images-30361//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01860_RIGHT_CC/07-20-2016-DDSM-NA-69131/1.000000-f

/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01886_RIGHT_CC/07-20-2016-DDSM-NA-04229/1.000000-full mammogram images-25726//1-1.dcm's resolution rescaled to (800, 600)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01883_LEFT_CC/07-20-2016-DDSM-NA-55434/1.000000-full mammogram images-42076//1-1.dcm is : BENIGN
Benign :/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/neg/6726.png
label :BENIGN
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01883_LEFT_CC/07-20-2016-DDSM-NA-55434/1.000000-full mammogram images-42076//1-1.dcm's resolution is (5888, 3664)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01883_LEFT_CC/07-20-2016-DDSM-NA-55434/1.000000-full mammogram images-42076//1-1.dcm's resolution rescaled to (800, 600)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01887_LEFT_CC/07-20-2016-DDSM-NA-60228/1.000000-full mammogram images-98521//1-1.dcm is : BENIGN
Benign :/home/careinfolab/FI

/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01981_RIGHT_MLO/07-20-2016-DDSM-NA-71482/1.000000-full mammogram images-25138//1-1.dcm's resolution rescaled to (800, 600)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01983_LEFT_MLO/07-20-2016-DDSM-NA-67431/1.000000-full mammogram images-43230//1-1.dcm is : MALIGNANT
Malignant : /home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/pos/6755.png
label :MALIGNANT
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01983_LEFT_MLO/07-20-2016-DDSM-NA-67431/1.000000-full mammogram images-43230//1-1.dcm's resolution is (6481, 3766)
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-Training_P_01983_LEFT_MLO/07-20-2016-DDSM-NA-67431/1.000000-full mammogram images-43230//1-1.dcm's resolution rescaled to (800, 600)
the last character is numeric. ANd is going to be continued!
the last character is numeric. ANd is going to be continued!
/home/careinfolab/StanfordMammogram/CBIS-DDSM/Mass-

In [9]:
neg_df = pd.DataFrame(NegativeCBISDF, columns=['empi_anon', 'file_path'])
pos_df = pd.DataFrame(PositiveCBISDF, columns=['empi_anon', 'file_path'])

In [10]:
###Metadata files are created for benign and malignant mammogram images.
neg_df.to_csv('/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/neg_empi_path.csv')
pos_df.to_csv('/home/careinfolab/FIR_Inchan/breast_CBISDDSM/images/800x600/benign_negative/pos_empi_path.csv')